# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


In [18]:
%idle_timeout 120
%glue_version 4.0
%worker_type G.1X
%number_of_workers 3
%%configure
{
    "--datalake-formats": "iceberg"
}

Current idle_timeout is 120 minutes.
idle_timeout has been set to 120 minutes.
Setting Glue version to: 4.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 3
Setting new number of workers to: 3
The following configurations have been updated: {'--datalake-formats': 'iceberg'}


In [1]:
import boto3
import sys
from awsglue.dynamicframe import DynamicFrame
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

spark.conf.set("spark.sql.catalog.iceberg_catalog", "org.apache.iceberg.spark.SparkCatalog")
spark.conf.set("spark.sql.catalog.iceberg_catalog.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
spark.conf.set("spark.sql.catalog.iceberg_catalog.warehouse", "s3://if4044-big-data-kel-4/iceberg/")
spark.conf.set("spark.sql.catalog.iceberg_catalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")


Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 3
Idle Timeout: 120
Session ID: 97459456-dbae-4de0-b5fe-e39f51a7fb26
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
--datalake-formats iceberg
Waiting for session 97459456-dbae-4de0-b5fe-e39f51a7fb26 to get into ready status...
Session 97459456-dbae-4de0-b5fe-e39f51a7fb26 has been created.



In [3]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg_catalog.db")

DataFrame[]


In [2]:
spark.table("iceberg_catalog.db.nation").show(5)
# spark.sql("SHOW NAMESPACES IN iceberg_catalog").show()
# print(spark.version)
# spark.conf.get("spark.sql.catalog.iceberg_catalog")


+-----------+---------+-----------+--------------------+
|N_NATIONKEY|   N_NAME|N_REGIONKEY|           N_COMMENT|
+-----------+---------+-----------+--------------------+
|          0|  ALGERIA|          0| haggle. carefull...|
|          1|ARGENTINA|          1|al foxes promise ...|
|          2|   BRAZIL|          1|y alongside of th...|
|          3|   CANADA|          1|eas hang ironic, ...|
|          4|    EGYPT|          4|y above the caref...|
+-----------+---------+-----------+--------------------+
only showing top 5 rows


In [7]:
def parquet_to_iceberg(s3_folder_uri: str, table_name: str, partition_by: list[str] | None = None):
    df = spark.read.parquet(f"{s3_folder_uri}/{table_name}.*.parquet")
    
    writer = df.writeTo(f"iceberg_catalog.db.{table_name}") \
               .tableProperty("format-version", "2")

    if partition_by is not None:
        writer = writer.partitionedBy(*partition_by)
        
    writer.createOrReplace()

In [5]:
parquet_location = "s3://if4044-big-data-kel-4/tpc-h-50gb-parquet"    

In [8]:
parquet_to_iceberg(f"{parquet_location}/CUSTOMER", "customer")
parquet_to_iceberg(f"{parquet_location}/LINEITEM", "lineitem")
parquet_to_iceberg(f"{parquet_location}/NATION", "nation")
parquet_to_iceberg(f"{parquet_location}/ORDERS", "orders")
parquet_to_iceberg(f"{parquet_location}/PART", "part")
parquet_to_iceberg(f"{parquet_location}/PARTSUPP", "partsupp")
parquet_to_iceberg(f"{parquet_location}/REGION", "region")
parquet_to_iceberg(f"{parquet_location}/SUPPLIER", "supplier")

In [10]:
spark.table("iceberg_catalog.db.customer").show(5)

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
|C_CUSTKEY|            C_NAME|           C_ADDRESS|C_NATIONKEY|        C_PHONE|C_ACCTBAL|C_MKTSEGMENT|           C_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
|        1|Customer#000000001|   IVhzIApeRb ot,c,E|         15|25-989-741-2988|   711.56|    BUILDING|to the even, regu...|
|        2|Customer#000000002|XSTf4,NCwDVaWNe6t...|         13|23-768-687-3665|   121.65|  AUTOMOBILE|l accounts. blith...|
|        3|Customer#000000003|        MG9kdTD2WBHm|          1|11-719-748-3364|  7498.12|  AUTOMOBILE| deposits eat sly...|
|        4|Customer#000000004|         XxVSJsLAGtn|          4|14-128-190-5944|  2866.83|   MACHINERY| requests. final,...|
|        5|Customer#000000005|KvpyuHCplrB84WgAi...|          3|13-750-942-6364|   794.47|   HOUSEHOLD|n accounts will h...|
+-------

In [11]:
spark.table("iceberg_catalog.db.lineitem").show(5)

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------+----------+--------------------+
|L_ORDERKEY|L_PARTKEY|L_SUPPKEY|L_LINENUMBER|L_QUANTITY|L_EXTENDEDPRICE|L_DISCOUNT|L_TAX|L_RETURNFLAG|L_LINESTATUS|L_SHIPDATE|L_COMMITDATE|L_RECEIPTDATE|L_SHIPINSTRUCT|L_SHIPMODE|           L_COMMENT|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------+----------+--------------------+
|   6290498|  6462217|   212242|           3|      12.0|       14146.68|      0.07| 0.05|           A|           F|1992-11-05|  1992-11-08|   1992-11-08|          NONE|      RAIL| slow excuses about |
|   6290498|   469060|   219061|           4|       5.0|         5145.2|       0.0| 0.05|           R|           F|1993-01-15|  1992-11-26|   1993-01-30|   COLLECT COD|     TRUCK|        e ideas. 

In [9]:
spark.table("iceberg_catalog.db.nation").show(5)

+-----------+---------+-----------+--------------------+
|N_NATIONKEY|   N_NAME|N_REGIONKEY|           N_COMMENT|
+-----------+---------+-----------+--------------------+
|          0|  ALGERIA|          0| haggle. carefull...|
|          1|ARGENTINA|          1|al foxes promise ...|
|          2|   BRAZIL|          1|y alongside of th...|
|          3|   CANADA|          1|eas hang ironic, ...|
|          4|    EGYPT|          4|y above the caref...|
+-----------+---------+-----------+--------------------+
only showing top 5 rows


In [12]:
spark.table("iceberg_catalog.db.orders").show(5)

+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
|O_ORDERKEY|O_CUSTKEY|O_ORDERSTATUS|O_TOTALPRICE|O_ORDERDATE|O_ORDERPRIORITY|        O_CLERK|O_SHIPPRIORITY|           O_COMMENT|
+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
| 102582913|  1026650|            O|   111074.28| 1996-06-04|       1-URGENT|Clerk#000008415|             0|ctions boost slyl...|
| 102582914|  4900319|            P|   149567.02| 1995-06-01|       1-URGENT|Clerk#000030552|             0|unts. slyly ironi...|
| 102582915|  2156444|            O|   236434.73| 1997-02-24|       3-MEDIUM|Clerk#000026208|             0| accounts. blithe...|
| 102582916|  3338122|            F|   105807.92| 1993-03-24|       1-URGENT|Clerk#000018865|             0|arefully final pi...|
| 102582917|  3811882|            O|   125658.05| 1996-06-15|       3-MEDIUM|Clerk#0000058

In [13]:
spark.table("iceberg_catalog.db.part").show(5)

+---------+--------------------+--------------+--------+--------------------+------+-----------+-------------+-------------------+
|P_PARTKEY|              P_NAME|        P_MFGR| P_BRAND|              P_TYPE|P_SIZE|P_CONTAINER|P_RETAILPRICE|          P_COMMENT|
+---------+--------------------+--------------+--------+--------------------+------+-----------+-------------+-------------------+
|        1|goldenrod lavende...|Manufacturer#1|Brand#13|PROMO BURNISHED C...|     7|  JUMBO PKG|        901.0|    ly. slyly ironi|
|        2|blush thistle blu...|Manufacturer#1|Brand#13| LARGE BRUSHED BRASS|     1|    LG CASE|        902.0|   lar accounts amo|
|        3|spring green yell...|Manufacturer#4|Brand#42|STANDARD POLISHED...|    21|  WRAP CASE|        903.0|egular deposits hag|
|        4|cornflower chocol...|Manufacturer#3|Brand#34|  SMALL PLATED BRASS|    14|   MED DRUM|        904.0|      p furiously r|
|        5|forest brown cora...|Manufacturer#3|Brand#32|STANDARD POLISHED...|    15

In [14]:
spark.table("iceberg_catalog.db.partsupp").show(5)

+----------+----------+-----------+-------------+--------------------+
|PS_PARTKEY|PS_SUPPKEY|PS_AVAILQTY|PS_SUPPLYCOST|          PS_COMMENT|
+----------+----------+-----------+-------------+--------------------+
|   8524289|     24290|       6913|       476.06|al pinto beans wa...|
|   8524289|    149307|       7951|       259.61|un blithely acros...|
|   8524289|    274324|       4259|       801.46|lly express reque...|
|   8524289|    399341|       6770|       705.46|he blithely ironi...|
|   8524290|     24291|        404|       654.73|ual dinos. blithe...|
+----------+----------+-----------+-------------+--------------------+
only showing top 5 rows


In [15]:
spark.table("iceberg_catalog.db.region").show(5)

+-----------+-----------+--------------------+
|R_REGIONKEY|     R_NAME|           R_COMMENT|
+-----------+-----------+--------------------+
|          0|     AFRICA|lar deposits. bli...|
|          1|    AMERICA|hs use ironic, ev...|
|          2|       ASIA|ges. thinly even ...|
|          3|     EUROPE|ly final courts c...|
|          4|MIDDLE EAST|uickly special ac...|
+-----------+-----------+--------------------+


In [16]:
spark.table("iceberg_catalog.db.supplier").show(5)

+---------+------------------+--------------------+-----------+---------------+---------+--------------------+
|S_SUPPKEY|            S_NAME|           S_ADDRESS|S_NATIONKEY|        S_PHONE|S_ACCTBAL|           S_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+--------------------+
|        1|Supplier#000000001| N kD4on9OM Ipw3,...|         17|27-918-335-1736|  5755.94|each slyly above ...|
|        2|Supplier#000000002|89eJ5ksX3ImxJQBvx...|          5|15-679-861-2259|  4032.68| slyly bold instr...|
|        3|Supplier#000000003|q1,G3Pj6OjIuUYfUo...|          1|11-383-516-1199|   4192.4|blithely silent r...|
|        4|Supplier#000000004|Bk7ah4CK8SYQTepEm...|         15|25-843-787-7479|  4641.08|riously even requ...|
|        5|Supplier#000000005|   Gcdm2rJRzl5qlTVzc|         11|21-151-690-3663|  -283.84|. slyly regular p...|
+---------+------------------+--------------------+-----------+---------------+---------+--------------------+
o